# MITSUI Commodity Prediction Challenge - Data Exploration

This notebook provides a comprehensive exploratory data analysis (EDA) of the competition data.

## Contents
1. Data Loading and Basic Information
2. Missing Data Analysis
3. Target Variable Analysis
4. Feature Distribution Analysis
5. Time Series Patterns
6. Cross-Market Correlations
7. Data Quality Assessment

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src to path for imports
sys.path.append('../src')
from utils.data_loader import DataLoader, load_competition_data

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Data Loading and Basic Information

In [ ]:
# Load all competition data
loader, data = load_competition_data('../')

# Extract dataframes for easier access
train_df = data['train']
train_labels_df = data['train_labels']
test_df = data['test']
target_pairs_df = data['target_pairs']
lagged_test_labels = data['lagged_test_labels']

print("Data loaded successfully!")
print(f"Training data: {train_df.shape}")
print(f"Training labels: {train_labels_df.shape}")
print(f"Test data: {test_df.shape}")
print(f"Target pairs: {target_pairs_df.shape}")
print(f"Lagged test labels: {len(lagged_test_labels)} files")

In [ ]:
# Get detailed data information
data_info = loader.get_data_info()

for name, info in data_info.items():
    print(f"\n=== {name.upper()} ===")
    print(f"Shape: {info['shape']}")
    print(f"Memory usage: {info['memory_usage_mb']:.2f} MB")
    print(f"Data types: {info['dtypes']}")
    print(f"Missing values: {info['missing_values']} ({info['missing_percentage']:.2f}%)")

In [ ]:
# Feature categories analysis
feature_categories = loader.get_feature_categories()

print("Feature Categories:")
total_features = 0
for category, features in feature_categories.items():
    print(f"{category.upper()}: {len(features)} features")
    total_features += len(features)
    
print(f"\nTotal features: {total_features}")

# Visualize feature distribution
fig, ax = plt.subplots(figsize=(10, 6))
categories = list(feature_categories.keys())
counts = [len(features) for features in feature_categories.values()]

bars = ax.bar(categories, counts, alpha=0.7)
ax.set_title('Number of Features by Category', fontsize=14, fontweight='bold')
ax.set_ylabel('Number of Features')
ax.set_xlabel('Feature Category')

# Add value labels on bars
for bar, count in zip(bars, counts):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 5,
            f'{count}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 2. Missing Data Analysis

In [ ]:
# Get missing data summaries
missing_summaries = loader.get_missing_data_summary()

# Analyze missing data in training features
print("=== TRAINING FEATURES MISSING DATA ===")
train_missing = missing_summaries['train']
features_with_missing = train_missing[train_missing > 0].sort_values(ascending=False)

if len(features_with_missing) > 0:
    print(f"Features with missing values: {len(features_with_missing)}")
    print("\nTop 10 features with most missing values:")
    print(features_with_missing.head(10))
    
    # Visualize missing data patterns
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Missing values by feature category
    missing_by_category = {}
    for category, features in feature_categories.items():
        category_missing = train_missing[features].sum()
        missing_by_category[category] = category_missing
    
    ax1.bar(missing_by_category.keys(), missing_by_category.values(), alpha=0.7)
    ax1.set_title('Missing Values by Feature Category')
    ax1.set_ylabel('Number of Missing Values')
    ax1.tick_params(axis='x', rotation=45)
    
    # Top features with missing values
    if len(features_with_missing) >= 10:
        top_missing = features_with_missing.head(10)
    else:
        top_missing = features_with_missing
    
    ax2.barh(range(len(top_missing)), top_missing.values, alpha=0.7)
    ax2.set_yticks(range(len(top_missing)))
    ax2.set_yticklabels([name.replace('_', '\n') if len(name) > 20 else name for name in top_missing.index])
    ax2.set_title('Top Features with Missing Values')
    ax2.set_xlabel('Number of Missing Values')
    
    plt.tight_layout()
    plt.show()
else:
    print("No missing values in training features!")

In [ ]:
# Analyze missing data in training labels
print("\n=== TRAINING LABELS MISSING DATA ===")
labels_missing = missing_summaries['train_labels']
targets_with_missing = labels_missing[labels_missing > 0].sort_values(ascending=False)

print(f"Targets with missing values: {len(targets_with_missing)}")
print(f"Total missing label values: {targets_with_missing.sum()}")

if len(targets_with_missing) > 0:
    # Missing data statistics
    print(f"\nMissing data statistics:")
    print(f"Min missing per target: {targets_with_missing.min()}")
    print(f"Max missing per target: {targets_with_missing.max()}")
    print(f"Mean missing per target: {targets_with_missing.mean():.1f}")
    print(f"Median missing per target: {targets_with_missing.median():.1f}")
    
    # Visualize missing patterns in targets
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Histogram of missing values
    ax1.hist(targets_with_missing.values, bins=20, alpha=0.7, edgecolor='black')
    ax1.set_title('Distribution of Missing Values per Target')
    ax1.set_xlabel('Number of Missing Values')
    ax1.set_ylabel('Number of Targets')
    
    # Missing percentage over time (if there's a time pattern)
    missing_by_date = train_labels_df.groupby('date_id').apply(
        lambda x: x.drop('date_id', axis=1).isnull().sum().sum()
    )
    
    ax2.plot(missing_by_date.index, missing_by_date.values, alpha=0.7)
    ax2.set_title('Missing Values Over Time')
    ax2.set_xlabel('Date ID')
    ax2.set_ylabel('Total Missing Values')
    
    plt.tight_layout()
    plt.show()

## 3. Target Variable Analysis

In [ ]:
# Analyze target pairs
print("=== TARGET PAIRS ANALYSIS ===")
print(f"Total targets: {len(target_pairs_df)}")
print(f"Unique lags: {sorted(target_pairs_df['lag'].unique())}")
print(f"Lag distribution:")
print(target_pairs_df['lag'].value_counts().sort_index())

# Analyze target types (single asset vs spreads)
target_pairs_df['is_spread'] = target_pairs_df['pair'].str.contains(' - ')
spread_count = target_pairs_df['is_spread'].sum()
single_count = len(target_pairs_df) - spread_count

print(f"\nTarget types:")
print(f"Single assets: {single_count}")
print(f"Spreads: {spread_count}")

# Show example targets
print(f"\nExample single asset targets:")
single_targets = target_pairs_df[~target_pairs_df['is_spread']]['pair'].head(5)
for i, target in enumerate(single_targets, 1):
    print(f"{i}. {target}")

print(f"\nExample spread targets:")
spread_targets = target_pairs_df[target_pairs_df['is_spread']]['pair'].head(5)
for i, target in enumerate(spread_targets, 1):
    print(f"{i}. {target}")

In [ ]:
# Analyze target value distributions
target_columns = [col for col in train_labels_df.columns if col.startswith('target_')]
sample_targets = target_columns[:12]  # Analyze first 12 targets

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

target_stats = []

for i, target in enumerate(sample_targets):
    # Get non-null values
    values = train_labels_df[target].dropna()
    
    if len(values) > 0:
        # Plot distribution
        axes[i].hist(values, bins=30, alpha=0.7, edgecolor='black')
        axes[i].set_title(f'{target}\n(n={len(values)})', fontsize=10)
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Frequency')
        
        # Calculate statistics
        stats = {
            'target': target,
            'count': len(values),
            'mean': values.mean(),
            'std': values.std(),
            'min': values.min(),
            'max': values.max(),
            'skew': values.skew(),
            'kurtosis': values.kurtosis()
        }
        target_stats.append(stats)

plt.suptitle('Target Value Distributions (First 12 Targets)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Display statistics table
stats_df = pd.DataFrame(target_stats)
print("\nTarget Statistics Summary:")
print(stats_df.round(6))

In [ ]:
# Analyze target correlations
print("=== TARGET CORRELATIONS ===")

# Calculate correlation matrix for a subset of targets (to avoid memory issues)
sample_targets_for_corr = target_columns[:50]  # First 50 targets
target_corr_matrix = train_labels_df[sample_targets_for_corr].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(target_corr_matrix, dtype=bool))
sns.heatmap(target_corr_matrix, mask=mask, cmap='RdBu_r', center=0,
            square=True, annot=False, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Target Correlation Matrix (First 50 Targets)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Find highly correlated target pairs
high_corr_pairs = []
for i in range(len(sample_targets_for_corr)):
    for j in range(i+1, len(sample_targets_for_corr)):
        corr = target_corr_matrix.iloc[i, j]
        if abs(corr) > 0.8 and not pd.isna(corr):
            high_corr_pairs.append({
                'target1': sample_targets_for_corr[i],
                'target2': sample_targets_for_corr[j],
                'correlation': corr
            })

if high_corr_pairs:
    print(f"\nHighly correlated target pairs (|corr| > 0.8): {len(high_corr_pairs)}")
    high_corr_df = pd.DataFrame(high_corr_pairs)
    print(high_corr_df.head(10))
else:
    print("\nNo highly correlated target pairs found (|corr| > 0.8)")

## 4. Feature Distribution Analysis

In [ ]:
# Analyze feature distributions by category
print("=== FEATURE DISTRIBUTION ANALYSIS ===")

# Sample features from each category for analysis
sample_features = {}
for category, features in feature_categories.items():
    # Sample up to 3 features from each category
    sample_size = min(3, len(features))
    sample_features[category] = features[:sample_size]
    
# Create subplots for feature distributions
fig, axes = plt.subplots(4, 3, figsize=(15, 16))

for cat_idx, (category, features) in enumerate(sample_features.items()):
    for feat_idx, feature in enumerate(features):
        if feat_idx < 3:  # Only plot up to 3 features per category
            values = train_df[feature].dropna()
            
            if len(values) > 0:
                axes[cat_idx, feat_idx].hist(values, bins=50, alpha=0.7, edgecolor='black')
                axes[cat_idx, feat_idx].set_title(f'{category.upper()}\n{feature}', fontsize=10)
                axes[cat_idx, feat_idx].set_xlabel('Value')
                axes[cat_idx, feat_idx].set_ylabel('Frequency')
                
                # Add basic statistics as text
                mean_val = values.mean()
                std_val = values.std()
                axes[cat_idx, feat_idx].axvline(mean_val, color='red', linestyle='--', alpha=0.7, label=f'Mean: {mean_val:.2f}')
                axes[cat_idx, feat_idx].legend(fontsize=8)

plt.suptitle('Feature Distributions by Category (Sample Features)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Statistical summary by feature category
print("\n=== FEATURE STATISTICS BY CATEGORY ===")

category_stats = {}
for category, features in feature_categories.items():
    category_data = train_df[features]
    
    stats = {
        'feature_count': len(features),
        'mean_range': (category_data.mean().min(), category_data.mean().max()),
        'std_range': (category_data.std().min(), category_data.std().max()),
        'missing_values': category_data.isnull().sum().sum(),
        'missing_percentage': (category_data.isnull().sum().sum() / category_data.size) * 100
    }
    category_stats[category] = stats
    
    print(f"\n{category.upper()}:")
    print(f"  Features: {stats['feature_count']}")
    print(f"  Mean range: {stats['mean_range'][0]:.2f} to {stats['mean_range'][1]:.2f}")
    print(f"  Std range: {stats['std_range'][0]:.2f} to {stats['std_range'][1]:.2f}")
    print(f"  Missing: {stats['missing_values']} ({stats['missing_percentage']:.2f}%)")

## 5. Time Series Patterns

In [ ]:
# Analyze time series patterns
print("=== TIME SERIES PATTERNS ===")

# Sample time series from each category
sample_ts_features = {}
for category, features in feature_categories.items():
    # Pick the first feature that has no missing values
    for feature in features:
        if train_df[feature].isnull().sum() == 0:
            sample_ts_features[category] = feature
            break
    if category not in sample_ts_features and features:  # If all have missing values, pick first
        sample_ts_features[category] = features[0]

# Plot time series
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, (category, feature) in enumerate(sample_ts_features.items()):
    if idx < 4:  # Only plot 4 categories
        ts_data = train_df[['date_id', feature]].dropna()
        
        axes[idx].plot(ts_data['date_id'], ts_data[feature], alpha=0.7)
        axes[idx].set_title(f'{category.upper()}\n{feature}', fontsize=12)
        axes[idx].set_xlabel('Date ID')
        axes[idx].set_ylabel('Value')
        axes[idx].grid(True, alpha=0.3)

plt.suptitle('Time Series Patterns by Category', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze target time series patterns
sample_target_features = target_columns[:4]  # First 4 targets

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, target in enumerate(sample_target_features):
    target_data = train_labels_df[['date_id', target]].dropna()
    
    axes[idx].plot(target_data['date_id'], target_data[target], alpha=0.7, color='red')
    axes[idx].set_title(f'{target}', fontsize=12)
    axes[idx].set_xlabel('Date ID')
    axes[idx].set_ylabel('Target Value')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].axhline(y=0, color='black', linestyle='-', alpha=0.5)

plt.suptitle('Target Time Series Patterns', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Data Quality Assessment

In [ ]:
# Comprehensive data quality assessment
print("=== DATA QUALITY ASSESSMENT ===")

# Data validation results
validation_results = loader.validate_data_consistency()
print("Data Consistency Checks:")
for check, result in validation_results.items():
    status = "✓ PASS" if result else "✗ FAIL"
    print(f"  {check}: {status}")

# Check for duplicate rows
print("\nDuplicate Row Analysis:")
train_duplicates = train_df.duplicated().sum()
test_duplicates = test_df.duplicated().sum() 
labels_duplicates = train_labels_df.duplicated().sum()
print(f"  Training data duplicates: {train_duplicates}")
print(f"  Test data duplicates: {test_duplicates}")
print(f"  Training labels duplicates: {labels_duplicates}")

# Check for infinite or extremely large values
print("\nExtreme Values Analysis:")
numeric_cols = train_df.select_dtypes(include=[np.number]).columns
inf_count = np.isinf(train_df[numeric_cols]).sum().sum()
very_large_count = (np.abs(train_df[numeric_cols]) > 1e10).sum().sum()
print(f"  Infinite values: {inf_count}")
print(f"  Very large values (>1e10): {very_large_count}")

# Check date_id consistency
print("\nDate ID Analysis:")
train_date_range = (train_df['date_id'].min(), train_df['date_id'].max())
test_date_range = (test_df['date_id'].min(), test_df['date_id'].max())
print(f"  Training date range: {train_date_range}")
print(f"  Test date range: {test_date_range}")
print(f"  Training data continuity: {len(train_df)} rows for {train_date_range[1] - train_date_range[0] + 1} date range")

# Data type consistency
print(f"\nData Types:")
print(f"  Training data types: {train_df.dtypes.value_counts().to_dict()}")
print(f"  Test data types: {test_df.dtypes.value_counts().to_dict()}")

## 7. Summary and Conclusions

In [ ]:
print("=== DATA EXPLORATION SUMMARY ===")
print()
print("📊 DATASET OVERVIEW:")
print(f"   • Training samples: {len(train_df):,}")
print(f"   • Test samples: {len(test_df):,}")
print(f"   • Features: {len(train_df.columns)-1:,}")
print(f"   • Targets: {len(target_columns):,}")
print()
print("🎯 TARGET CHARACTERISTICS:")
print(f"   • Single asset targets: {single_count}")
print(f"   • Spread targets: {spread_count}")
print(f"   • Targets with missing data: {len(targets_with_missing)}")
print()
print("🔍 FEATURE BREAKDOWN:")
for category, features in feature_categories.items():
    print(f"   • {category.upper()}: {len(features)} features")
print()
print("⚠️  DATA QUALITY ISSUES:")
print(f"   • Features with missing values: {len(features_with_missing)}")
print(f"   • Train/test feature mismatch: {'Yes' if not validation_results.get('train_test_features_match', True) else 'No'}")
print(f"   • Duplicate rows: {train_duplicates + test_duplicates + labels_duplicates}")
print()
print("🚀 KEY INSIGHTS:")
print("   • Data spans time series with sequential date_ids")
print("   • Mixed prediction task: single assets + spreads")
print("   • Significant missing data patterns, especially in JPX features")
print("   • Target values are normalized returns (roughly -0.12 to +0.11)")
print("   • US Stock features dominate the feature space (475/557)")
print()
print("📋 NEXT STEPS:")
print("   1. Implement robust missing data handling strategy")
print("   2. Create separate models for single assets vs spreads")
print("   3. Focus on time series feature engineering")
print("   4. Consider feature selection due to high dimensionality")
print("   5. Implement proper time series cross-validation")

In [ ]:
# Save key findings to file
findings = {
    'dataset_info': data_info,
    'feature_categories': {k: len(v) for k, v in feature_categories.items()},
    'missing_data_summary': {
        'features_with_missing': len(features_with_missing),
        'targets_with_missing': len(targets_with_missing)
    },
    'target_analysis': {
        'single_asset_targets': single_count,
        'spread_targets': spread_count,
        'total_targets': len(target_columns)
    },
    'data_quality': validation_results
}

# This would save to a JSON file in a real implementation
print("Key findings documented for further analysis.")
print("\n📝 EDA Complete! Ready for preprocessing and feature engineering.")